In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

In [4]:
module = model.conv1
print(module)
print(list(module.named_parameters()))

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
[('bias', Parameter containing:
tensor([ 0.0108, -0.0079,  0.1052,  0.0651,  0.0797, -0.2543],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.2941, -0.0804, -0.3134],
          [-0.0099,  0.0290,  0.0828],
          [ 0.3250, -0.1015,  0.1196]]],


        [[[-0.1949, -0.1495,  0.2200],
          [-0.0327, -0.0476, -0.2516],
          [ 0.2741, -0.2347, -0.0397]]],


        [[[ 0.1081, -0.2961, -0.1651],
          [-0.1870, -0.3307,  0.1295],
          [ 0.2603, -0.3210, -0.3223]]],


        [[[-0.2598, -0.3015,  0.0519],
          [-0.2764,  0.1571, -0.1859],
          [-0.0807,  0.1851,  0.2222]]],


        [[[-0.2159, -0.0311, -0.0404],
          [-0.3136, -0.0617, -0.1504],
          [ 0.2281,  0.0406,  0.0801]]],


        [[[-0.0706, -0.2271, -0.0031],
          [-0.0516, -0.1938,  0.0482],
          [ 0.1937,  0.2082, -0.0860]]]], requires_grad=True))]


In [3]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [5]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 1., 0.],
          [0., 0., 0.],
          [1., 0., 0.]]],


        [[[1., 1., 1.],
          [1., 1., 0.],
          [1., 1., 1.]]],


        [[[1., 0., 1.],
          [1., 1., 1.],
          [1., 1., 0.]]],


        [[[1., 0., 1.],
          [1., 0., 1.],
          [0., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [0., 1., 1.]]],


        [[[1., 0., 1.],
          [0., 1., 0.],
          [1., 1., 1.]]]]))]


In [7]:

#the weights are changed to zero by the pruning at random by 30%. We can see which weights are masked by printing named buffers
print(module.weight)

print(module._forward_pre_hooks)

tensor([[[[ 0.2941, -0.0804, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.3250, -0.0000,  0.0000]]],


        [[[-0.1949, -0.1495,  0.2200],
          [-0.0327, -0.0476, -0.0000],
          [ 0.2741, -0.2347, -0.0397]]],


        [[[ 0.1081, -0.0000, -0.1651],
          [-0.1870, -0.3307,  0.1295],
          [ 0.2603, -0.3210, -0.0000]]],


        [[[-0.2598, -0.0000,  0.0519],
          [-0.2764,  0.0000, -0.1859],
          [-0.0000,  0.1851,  0.2222]]],


        [[[-0.2159, -0.0311, -0.0404],
          [-0.3136, -0.0617, -0.1504],
          [ 0.0000,  0.0406,  0.0801]]],


        [[[-0.0706, -0.0000, -0.0031],
          [-0.0000, -0.1938,  0.0000],
          [ 0.1937,  0.2082, -0.0860]]]], grad_fn=<MulBackward0>)
OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x000002C114E7BA90>)])


In [13]:
prune.l1_unstructured(module, name="bias", amount=4)

TypeError: cannot assign 'torch.FloatTensor' object to parameter 'bias_orig' (torch.nn.Parameter or None required)

In [14]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to
# 50% (3 out of 6) of the channels, while preserving the action of the
# previous mask.
print(module.weight)

tensor([[[[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[-0.1949, -0.1495,  0.2200],
          [-0.0327, -0.0476, -0.0000],
          [ 0.2741, -0.2347, -0.0397]]],


        [[[ 0.1081, -0.0000, -0.1651],
          [-0.1870, -0.3307,  0.1295],
          [ 0.2603, -0.3210, -0.0000]]],


        [[[-0.2598, -0.0000,  0.0519],
          [-0.2764,  0.0000, -0.1859],
          [-0.0000,  0.1851,  0.2222]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000]]]], grad_fn=<MulBackward0>)


In [15]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))  # pruning history in the container

[<torch.nn.utils.prune.RandomUnstructured object at 0x000002C114E7BA90>, <torch.nn.utils.prune.LnStructured object at 0x000002C12CE5EA30>]


In [16]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [17]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.2941, -0.0804, -0.3134],
          [-0.0099,  0.0290,  0.0828],
          [ 0.3250, -0.1015,  0.1196]]],


        [[[-0.1949, -0.1495,  0.2200],
          [-0.0327, -0.0476, -0.2516],
          [ 0.2741, -0.2347, -0.0397]]],


        [[[ 0.1081, -0.2961, -0.1651],
          [-0.1870, -0.3307,  0.1295],
          [ 0.2603, -0.3210, -0.3223]]],


        [[[-0.2598, -0.3015,  0.0519],
          [-0.2764,  0.1571, -0.1859],
          [-0.0807,  0.1851,  0.2222]]],


        [[[-0.2159, -0.0311, -0.0404],
          [-0.3136, -0.0617, -0.1504],
          [ 0.2281,  0.0406,  0.0801]]],


        [[[-0.0706, -0.2271, -0.0031],
          [-0.0516, -0.1938,  0.0482],
          [ 0.1937,  0.2082, -0.0860]]]], requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.0108, -0.0079,  0.1052,  0.0651,  0.0797, -0.2543],
       requires_grad=True))]


In [18]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([ 0.0108, -0.0079,  0.1052,  0.0651,  0.0797, -0.2543],
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[-0.1949, -0.1495,  0.2200],
          [-0.0327, -0.0476, -0.0000],
          [ 0.2741, -0.2347, -0.0397]]],


        [[[ 0.1081, -0.0000, -0.1651],
          [-0.1870, -0.3307,  0.1295],
          [ 0.2603, -0.3210, -0.0000]]],


        [[[-0.2598, -0.0000,  0.0519],
          [-0.2764,  0.0000, -0.1859],
          [-0.0000,  0.1851,  0.2222]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000]]]], requires_grad=True))]


In [20]:
new_model = LeNet()
for name, module in new_model.named_modules():
  
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.1)
 
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.2)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


In [21]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 3.70%
Sparsity in conv2.weight: 8.56%
Sparsity in fc1.weight: 21.92%
Sparsity in fc2.weight: 12.67%
Sparsity in fc3.weight: 10.95%
Global sparsity: 20.00%


In [22]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask
    
    
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module



model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
